In [10]:
import sys
# Add the ptdraft folder path to the sys.path list
sys.path.append('../src/')

import katodata as kd
reload(kd)
data = kd.load()

data.

/Users/akivalipshitz/Developer/neuronal-analysis/data/wbdata/


,0,1,2,3,4
FlNm,TS20140715f_lite-1_punc-31_NLS3_3eggs_56um_1mM...,TS20140926d_lite-1_punc-31_NLS3_RIV_2eggs_1mMT...,TS20141221b_THK178_lite-1_punc-31_NLS3_6eggs_1...,TS20140715e_lite-1_punc-31_NLS3_2eggs_56um_1mM...,TS20140905c_lite-1_punc-31_NLS3_AVHJ_0eggs_1mM...
NeuronIds,0 1 0 None None 1 None ...,0 1 0 None None 1 None...,0 1 0 None None 1 None ...,0 1 2 0 None None Non...,0 1 0 NaN NaN 1 NaN Na...
deltaFOverF,"[[0.135913490458, 0.122698371026, 0.1273345364...","[[-0.0888920088322, -0.130398703723, -0.080847...","[[-0.15780023438, -0.119564953505, -0.18563226...","[[0.122448621446, 0.118398218526, 0.1275767208...","[[0.150902719177, -0.039497701204, 0.045866099..."
deltaFOverF_bc,"[[0.0813810375536, 0.068289481621, 0.073049190...","[[0.11694110675, 0.0756448955127, 0.1254065656...","[[0.102359940819, 0.141055885003, 0.0754484136...","[[0.0686293468348, 0.0647636490109, 0.07412679...","[[0.230656610926, 0.0403072988817, 0.125722194..."
deltaFOverF_deriv,"[[-0.000551819232698, -0.000551818672615, -0.0...","[[0.0062459423014, 0.00624604574426, 0.0062471...","[[-0.000489274045825, -0.000489858006098, -0.0...","[[0.000883089395239, 0.000883089373364, 0.0008...","[[-0.0309526801435, -0.0309354939679, -0.02839..."
tv,"[[0.0, 0.344607530313, 0.689215060625, 1.03382...","[[0.0, 0.326185442465, 0.652370884929, 0.97855...","[[0.0, 0.357497517378, 0.714995034757, 1.07249...","[[0.0, 0.344277972585, 0.688555945171, 1.03283...","[[0.0, 0.35305655443, 0.706113108859, 1.059169..."


In [8]:
data[0]['NeuronIds']

,0,1
0,None,None
1,None,None
2,None,None
3,None,None
4,None,None
5,None,None
6,None,None
7,None,None
8,None,None
9,None,None
